<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<h1 align="center">Open Data QnA - Chat with your SQL Database</h1> 

---

This notebook first walks through the Vector Store Setup needed for running the Open Data QnA application. 

Currently supported Source DBs are: 
- PostgreSQL on Google Cloud SQL 
- BigQuery

Furthermore, the following vector stores are supported 
- pgvector on PostgreSQL 
- BigQuery vector


The setup part covers the following steps: 
> 1. Configuration: Intial GCP project, IAM permissions, Environment  and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)

> 3. Setting up firestore for persisting the session history for multiturn


Afterwards, you will be able to run the Open Data QnA Pipeline to generate SQL queries and answer questions over your data source. 

### 📒 Using this interactive notebook

If you have not used this IDE with jupyter notebooks it will ask for installing Python + Jupyter extensions. Please go ahead install them

Click the **run** icons ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

# 🚧 **0. Prerequisites**

Make sure that Google Cloud CLI is installed before moving to the next cell! You can refer to the link below for guidance

Installation Guide: https://cloud.google.com/sdk/docs/install

##  **0.1. Setup Poetry Environment and Setup GCP Project** 

### 💻 **Install Code Dependencies (Create and setup venv)**
Install the dependencies by runnign the poetry commands below 

Note: Below command runs with default Python Kernel and we will change that to Kernel from venv after this execution below

In [ ]:
# Install poetry
! pip uninstall poetry -y
! pip install poetry --quiet

#Run the poetry commands below to set up the environment
!poetry lock #resolve dependecies (also auto create poetry venv if not exists)
!poetry install --quiet #installs dependencies
!poetry env info #Displays the evn just created and the path to it

### 📌 **Important Step: Activate your virtual environment and authenticate with Google Cloud CLI**

#### **All commands in this cell to be run on Terminal**


Chose the relevant instructions based on where you are running this notebook


**For IDEs like Cloud Shell Editor, VS Code**

Once the venv created either in the local directory or in the cache directory. Open the terminal on the same machine where your notebooks are running and start running the below commands.


```
poetry shell #this command should activate your venv and you should see it enters into the venv

##inside the activated venv shell []

gcloud auth login

gcloud services enable \
    serviceusage.googleapis.com \
    cloudresourcemanager.googleapis.com --project <<Enter Project Id>>

```
For IDEs adding Juypter Extensions will automatically give you option to change the kernel. If not, manually select the python interpreter in your IDE (The exact is shown in the above cell. Path would look like e.g. /home/admin_/Talk2Data/.venv/bin/python or ~cache/user/Talk2Data/.venv/bin/python)


**For Jupyter Lab or Jupyter Environments on Workbench etc**


```
poetry shell #this command should activate your venv and you should see it enters into the venv

##inside the activated venv shell []

#If you are running on Worbench instance where the service account used has required permissions to run this solution you can skip the below gcloud auth commands and get to next kernel creation section

gcloud auth application-default login
gcloud services enable \
    serviceusage.googleapis.com \
    cloudresourcemanager.googleapis.com --project <<Enter Project Id>>
gcloud auth application-default set-quota-project <<Enter Project Id for using resources>>

```

Create Kernel for with the envrionment created

```

pip install jupyter

ipython kernel install --name "openqna-venv" --user 

```

Restart your kernel or close the exsiting notebook and open again, you should now see the "openqna-venv" in the kernel drop down

**What did we do here?**

* Created Application Default Credentials to use for the code
* Added venv to kernel to select for runningt the notebooks (For standalone Jupyter setups like Workbench etc)

### **Change your Kernel to the above created .venv**

### Set Python Module Path to Root

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

### 🔗 **Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [2]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = "three-p-o"

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

#If errors out for authentication restart the kernel and start from the previous cell

Updated property [core/project].
Project has been set to three-p-o


### ⚙️ **Enable Required API Services in the GCP Project**

In [3]:
#Enable all the required APIs for the Open Data QnA solution

!gcloud services enable \
  cloudapis.googleapis.com \
  compute.googleapis.com \
  iam.googleapis.com \
  run.googleapis.com \
  sqladmin.googleapis.com \
  aiplatform.googleapis.com \
  bigquery.googleapis.com \
  firestore.googleapis.com --project {PROJECT_ID}

Operation "operations/acat.p2-978842762722-93441efe-e4b8-43c9-9578-b4652dc1d301" finished successfully.


## **Configure your inputs for the environments**

This section assumes that a datasource is already set up in your GCP project. If a datasource has not been set up, use the notebooks below to copy a public data set from BigQuery to Cloud SQL or BigQuery on your GCP project


Enabled Data Sources:
* PostgreSQL on Google Cloud SQL (Copy Sample Data: [0_CopyDataToCloudSqlPG.ipynb](0_CopyDataToCloudSqlPG.ipynb))
* BigQuery (Copy Sample Data: [0_CopyDataToBigQuery.ipynb](0_CopyDataToBigQuery.ipynb))

Enabled Vector Stores:
* pgvector on PostgreSQL 
* BigQuery vector


### 🤔 **Choose Data Source and Vector Store**

**Sources to connect**
- This solution lets you setup multiple data source at the same time.
- You can group multiple tables from different datasets or schema into a grouping and provide the details
- If your dataset/schema has many tables and you want to run the solution against few you should specifically choose a group for that tables only

**Format for data_source_list.csv**

**source | user_grouping | schema | table**

**source** - Supported Data Sources. #Options: bigquery , cloudsql-pg

**user_grouping** - Logical grouping or use case name for tables from same or different schema/dataset. When left black it default to the schema value in the next column

**schema** - schema name for postgres or dataset name in bigquery 

**table** - name of the tables to run the solutions against. Leave this column blank after filling schema/dataset if you want to run solution for whole dataset/schema


Fill out the parameters and configuration settings below. 
These are the parameters for setting configurations for the vector store tables to be created. 

Additionally, you can specify whether you have and want to use known-good-queries for the pipeline run and whether you want to enable logging.

**Known good queries:** if you have known working user question <-> SQL query pairs, you can put them into the file `scripts/known_good_sql.csv`. This will be used as a caching layer and for in-context learning: If an exact match of the user question is found in the vector store, the pipeline will skip SQL Generation and output the cached SQL query. If the similarity score is between 90-100%, the known good queries will be used as few-shot examples by the SQL Generator Agent. 

**Logging:** you can enable logging. If enabled, a dataset is created in Big Query in your project, which will store the logging table and save information from the pipeline run in the logging table. This is especially helpful for debugging.

**use_column_samples:** you can enable use column samples flag to let the pipeline select sample values of the columns from the source database. In some specific usecase where we need to understand the format or case sensitivity of the values this flag help LLM to have better understanding. Though this is one time setup, please be aware that turning this on mean getting samples from each column and it can be an expensive operation when there are lot many columns.

In [3]:
#[CONFIG]
embedding_model = 'vertex' # Options: 'vertex' or 'vertex-lang'
description_model = 'gemini-1.5-pro' # 'gemini-1.0-pro', 'gemini-1.5-pro', 'text-bison-32k', 'gemini-1.5-flash'
vector_store = 'bigquery-vector' # Options: 'bigquery-vector', 'cloudsql-pgvector'
logging = True # True or False 
kgq_examples = True # True or False
use_column_samples = True #True or False

#[GCP]
project_id = PROJECT_ID

#[PGCLOUDSQL]
# Default values for pgvector setup, change only if needed
pg_region = 'us-central1'
pg_instance = 'pg15-opendataqna'
pg_database = 'opendataqna-db'
pg_user = 'pguser'
pg_password = 'pg123'

#[BIGQUERY]
# Name for the BQ dataset created for bigquery-vector and/or logging. Change names only if needed.
bq_dataset_region = 'us-central1'
bq_opendataqna_dataset_name = 'opendataqna'
bq_log_table_name = 'audit_log_table'

#Details for firestore to store the chat session history
firestore_region='us-central1'
## firestore_database is named as 'opendataqna-session-logs' (This is designed to not be customizable as the setup includes creation of composite indexes from backend)

Quick input verifications below:

In [6]:

# Input verification - Vector Store
assert vector_store in {'bigquery-vector', 'cloudsql-pgvector'}, "⚠️ Invalid VECTOR_STORE. Must be 'bigquery-vector' or 'cloudsql-pgvector'"

# Input verification - Firestore Region
assert firestore_region, "⚠️ Provide firestore region name"

if logging: 
    assert bq_log_table_name, "⚠️ Please provide a name for your log table if you want to use logging"

if vector_store == 'bigquery':
    assert bq_dataset_region, "⚠️ Please provide the Data Set Region"
    assert bq_opendataqna_dataset_name, "⚠️ Please provide the name of the logging/vector store dataset on Bigquery"

elif vector_store == 'cloudsql-pg':
    assert pg_region, "⚠️ Please provide Region of the Cloud SQL Instance"
    assert pg_instance, "⚠️ Please provide the name of the Cloud SQL Instance"
    assert pg_database, "⚠️ Please provide the name of the PostgreSQL Database on the Cloud SQL Instance"
    assert pg_user, "⚠️ Please provide a username for the Cloud SQL Instance"
    assert pg_password, "⚠️ Please provide the Password for the PG_USER"


### 💾 **Save Configuration to File** 
Save the configurations set in this notebook to  `config.ini`. The parameters from this file are used in notebooks and in various modeules in the repo

In [7]:
from scripts import save_config

save_config(embedding_model, description_model, vector_store, logging, kgq_examples, use_column_samples, PROJECT_ID,
            pg_region, pg_instance, pg_database, pg_user, pg_password, 
            bq_dataset_region, bq_opendataqna_dataset_name, bq_log_table_name,firestore_region)

current dir:  /home/admin_/Open_Data_QnA/notebooks
All configuration paramaters saved to file!


# **1. Vector Store Setup** (Run once)
---

This section walks through the Vector Store Setup needed for running the Open Data QnA application. 

It covers the following steps: 
> 1. Configuration: Environment and Databases setup including logging on Bigquery for analytics

> 2. Creation of Table, Column and Known Good Query Embeddings in the Vector Store  for Retreival Augmented Generation(RAG)




## ⚙️ **1.1 Database Setup for Vector Store: CloudSQL-pgvector**

Create PostgreSQL Instance on CloudSQL if 'cloudsql-pgvector' is chosen as vector store

Note that a PostgreSQL Instance on CloudSQL already exists if 'cloudsql-pg' is the data source. PostgreSQL Instance is created only if a different data store is chosen.

The cell will also create a dataset to store the log table on Big Query, **if** logging is enabled.

In [12]:
from env_setup import create_vector_store
# Setup vector store for embeddings
create_vector_store()  


Initializing environment setup.
Loading configurations from config.ini file.
Vector Store source set to:  bigquery-vector
Logging is enabled
Vector store set to 'bigquery-vector'
Generating Big Query dataset opendataqna
Destination Dataset exists


##  **1.2 Create Embeddings in Vector Store for RAG** 

### 🖋️ **Create Table and Column Embeddings**

In this step, table and column metadata is retreived from the data source and embeddings are generated for both

In [4]:
from env_setup import get_embeddings

# Generate embeddings for tables and columns
table_schema_embeddings, col_schema_embeddings = get_embeddings()


current dir:  /home/admin_/Open_Data_QnA/notebooks
root_dir set to: /home/admin_/Open_Data_QnA


I0000 00:00:1721717874.012655    9216 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


Generating embeddings from source db schemas
Source Found at Path :: /home/admin_/Open_Data_QnA/scripts/data_source_list.csv
The Embeddings are extracted for the below combinations
     source        schema                                              table
0  bigquery   imdb_people  [name_basics, title_basics, title_crew, title_...
1  bigquery  imdb_ratings                           [reviews, title_ratings]



Generated table description for imdb_people.name_basics
Generated table description for imdb_people.title_crew
Generated table description for imdb_people.title_basics
Generated table description for imdb_people.title_principals
LLM generated 4 Table Descriptions



LLM generated 0 Column Descriptions



Generated table description for imdb_ratings.reviews
Generated table description for imdb_ratings.title_ratings
LLM generated 2 Table Descriptions



LLM generated 0 Column Descriptions
Table and Column embeddings are created


### 💾 **Save the Table and Column Embeddings in the Vector Store**
The table and column embeddings created in the above step are save to the Vector Store chosen

In [13]:
from env_setup import store_embeddings

# Store table/column embeddings (asynchronous)
await(store_embeddings(table_schema_embeddings, col_schema_embeddings))


Storing embeddings back to the vector store.
Table and Column embeddings are saved to vector store


# **2. Firestore Database Setup**
---

This section walks through setting up the firestore DB to store the session history of the conversation for multiturn

It covers the following steps: 
> 1. Creation Firestore Database



In [14]:
from env_setup import create_firestore_db

create_firestore_db(firestore_region)

Found existing Firestore database with this name already!


## 🗄️ **3. Load Known Good SQL into Vector Store**
Known Good Queries are used to create query cache for Few shot examples. Creating a query cache is highly recommended for best outcomes! 

The following cell will load the Natural Language Question and Known Good SQL pairs into our Vector Store. There pairs are loaded from `known_good_sql.csv` file inside scripts folder. If you have your own Question-SQL examples, curate them in .csv file before running the cell below. 

If no Known Good Queries are available at this time to create query cache, you can use [3_LoadKnownGoodSQL.ipynb](3_LoadKnownGoodSQL.ipynb) to load them later. 



#### Format of the Known Good SQL File (known_good_sql.csv)

prompt | sql | user_grouping [3 columns]

prompt ==> User Question 

sql ==> SQL for the user question (Note that the sql should enclosed in quotes and only in single line. Please remove the line  break)

user_grouping ==>This name should exactly  match the grouping name you mentioned while creating vector store

In [5]:
from env_setup import create_kgq_sql_table, store_kgq_sql_embeddings

# Create table for known good queries (if enabled)
await(create_kgq_sql_table()) 

# Store known good query embeddings (if enabled)
await(store_kgq_sql_embeddings())  


Creating kgq table in vector store.
Reading contents of known_good_sql.csv
Storing kgq embeddings in vector store table.
 LIMIT 1Y midyear_population DESC.midyear_population


BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/three-p-o/queries?prettyPrint=false: Syntax error: Unclosed string literal at [3:21]

### 🥁 If all the above steps are executed suucessfully, the following should be set up:

* GCP project and all the required IAM permissions

* Environment to run the solution

* Data source and Vector store for the solution

__________________________________________________________________________________________________________________